# Langchain: Use cases how to connect external data to LLMs
### (see manual https://python.langchain.com/docs/ for more details)

In [1]:
# Define text colors
green = "\033[0;32m"
white = "\033[0;39m"
cyan  = "\033[36m"

In [2]:
template_0 = """
You are a friendly chatbot assistant that responds in a conversational
manner to users questions. Keep the answers short, unless specifically
asked by the user to elaborate on something.

Question: {question}

Answer:"""

In [3]:
from langchain import PromptTemplate
prompt_0 = PromptTemplate(template=template_0, input_variables=["question"])

## Local LLM *(e.g., GTP4all)*

In [13]:
from langchain import LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_local = GPT4All(
    model='../Models/ggml-model-gpt4all-falcon-q4_0.bin',
    callbacks=[StreamingStdOutCallbackHandler()]
)

Found model file at  ../Models/ggml-model-gpt4all-falcon-q4_0.bin
falcon_model_load: loading model from '../Models/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


In [29]:
question_2 = input("Question? ")
llm_local(question_2)



The city of Paris was founded on May 24, 1252.

'\nThe city of Paris was founded on May 24, 1252.'

## Remote LLM via API call *(e.g., OpenAI )*

### New approach for "gpt-3.5-turbo"and "gpt-4" models (using openai.ChatCompletion method)

In [6]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# model_name = "gpt-3.5-turbo"
model_name = "gpt-4"

llm = ChatOpenAI(model_name=model_name, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [40]:
query = "Which language is used to create ChatGPT?"

In [41]:
response = llm([
    SystemMessage(content="You are friendly AI assistant"),
    HumanMessage(content=query)
])

print(response)
print(f"{green}{query}")
print(f"{cyan}ChatGPT: {white}{response.content}")

content="ChatGPT is created using Python programming language. It's based on the GPT (Generative Pretrained Transformer) model developed by OpenAI. The model itself is trained on a variety of internet text, but when it comes to refining the model or 'fine-tuning' it, this is done using Python." additional_kwargs={} example=False
Which language is used to create ChatGPT?
ChatGPT: ChatGPT is created using Python programming language. It's based on the GPT (Generative Pretrained Transformer) model developed by OpenAI. The model itself is trained on a variety of internet text, but when it comes to refining the model or 'fine-tuning' it, this is done using Python.


## Using prompt template

In [7]:
# Creating a prompt template and running the LLM chain
from langchain import PromptTemplate, LLMChain

template = "What are the top {n} resources to learn {topic} ?"
prompt = PromptTemplate(template=template,input_variables=['n','topic'])

input = {'n':3,'topic':'Python programming'}

In [8]:

chain = LLMChain(llm=llm,prompt=prompt)
print(chain.run(input))

1. Codecademy: This is an online interactive platform that offers free coding classes in 12 different programming languages including Python. It's a great place for beginners as it provides an interactive learning environment where you can write code in the browser and see the result instantly.

2. Coursera: Coursera offers a wide range of Python courses from top universities such as University of Michigan and Stanford. These courses cover everything from Python basics to data science and machine learning.

3. Python.org: The official Python website has its own free learning resources and tutorials for beginners and advanced users. You can also find a list of other helpful resources, including books and other websites.


## Connecting to SQL Database

In [5]:
db_file = 'Input/books.db'
db_uri = 'sqlite:///' + db_file

In [6]:
# Setup database

from langchain.utilities import SQLDatabase
db = SQLDatabase.from_uri(db_uri)

In [7]:
# setup LLM

from langchain.chat_models import ChatOpenAI
import os


llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv('OPENAI_API_KEY'), temperature=0)

In [8]:
# Setup the database chain
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db)

In [40]:
# Create Query template
QUERY = """
Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return all requested  data in following format:

QUESTION: Initial question here
SQL QUERY: SQL Query to run
SQL RESULT: Result of the SQLQuery execution
ANSWER: Answer in natural language

{question}
"""

In [37]:
# Function to generate prompt
def get_prompt():
    '''Prompt-generation function, Receives a keyborad input, sends a request to LLM and returns an answer''' 
    
    # Define text colors
    green = "\033[0;32m"
    white = "\033[0;39m"
    cyan  = "\033[36m"
    print("Type 'exit' to quit")

    while True:
        prompt = input("Enter a prompt: ")

        if prompt.lower() == 'exit':
            print('Exiting...')
            break
        else:
            try:
                question = QUERY.format(question=prompt)
                print(f"{green}{prompt}")
                print(f"{cyan}ChatGPT: {white}{db_chain.run(question)}")
            except Exception as e:
                print(e)


In [38]:
get_prompt()

Type 'exit' to quit
How many tables are in total?
ChatGPT: Final answer here: There are 6 tables in total.
Please return SQL query
ChatGPT: SQLQuery: SELECT * FROM Money LIMIT 5
incorrect. Try again
ChatGPT: incorrect. Try again
Think step by step
ChatGPT: SQLQuery: SELECT * FROM Money LIMIT 5;

ChatGPT: SQLQuery: SELECT * FROM Money LIMIT 5;
Exiting...


In [15]:
prompt_1 = "Add new author 'Smith, John' to the position 12"

In [16]:
db_chain.run(QUERY.format(question=prompt_1))

'Final answer here'

In [17]:
prompt_1 = "List all the authors"

In [18]:
prompt_1

'List all the authors'

In [20]:
print(db_chain.run(QUERY.format(question=prompt_1)))

Final answer here: Paul Deitel, Harvey Deitel, Abbey Deitel


In [ ]:
import sqlite3
conn = sqlite3.connect(db_file)

In [42]:
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

In [43]:
# Create the tasks table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS tasks
             (id SERIAL PRIMARY KEY,
             task TEXT NOT NULL,
             completed BOOLEAN,
             due_date DATE,
             completion_date DATE,
             priority INTEGER)''')

In [44]:
# Insert sample tasks into the tasks table
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Complete the web page design', True, '2023-05-01', '2023-05-03', 1)")


In [45]:
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Create login and signup pages', True, '2023-05-03', '2023-05-05', 2)")

In [46]:
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Product management', False, '2023-05-05', '2023-05-05', 3)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Cart and wishlist creation', False, '2023-05-08', '2023-05-05', 4)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES  ('Payment gateway integration', False, '2023-05-10', '2023-05-05', 5)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Order management', False, '2023-05-10', '2023-05-05', 6)")


In [47]:
conn.commit()

In [48]:
conn.close()

In [21]:
text = "Input a question: "

In [22]:
input(text)

'How many tableas are in total?'

In [23]:
get_prompt()

Type 'exit' to quit
Final answer here: There are 6 tables in total.
Final answer here: The tables in the database are 'sqlite_sequence', 'authors', 'titles', 'author_ISBN', 'tasks', and 'Money'.
Final answer here: The only table in the database with a corresponding entry in the sqlite_sequence table is the "authors" table, with a value of 12.
Exiting...
